In [8]:
from typing import Annotated 
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages # ----> reducer
from dotenv import load_dotenv
from langgraph.prebuilt import ToolNode, tools_condition
import os

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
os.environ["GOOGLE_API_KEY"] = google_api_key
os.environ["TAVILY_API_KEY"] = tavily_api_key 
from langchain_tavily import TavilySearch

tool = TavilySearch(
    max_results=2
    #topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [12]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("google_genai:gemini-2.0-flash")

E0000 00:00:1760355444.813519   38629 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [17]:
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers.
    Args:
        a (int): The first number.
        b (int): The second number.
    Returns:
        int: The product of the two numbers.
    """

In [ ]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool, multiply]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [21]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "Give me a summary of the latest news about AI and multiply 5 and 6."
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

/home/ahmad-faisal/miniconda3/envs/lang/lib/python3.12/site-packages/langchain_google_genai/chat_models.py:2007: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


User: Give me a summary of the latest news about AI and multiply 5 and 6.
Assistant: 
Assistant: null
Assistant: OK. Here's a summary of the latest news about AI: According to Reuters, stay up-to-date on artificial intelligence news, covering AI breakthroughs, technology trends, regulation, ethics, business and global impact. Also, Artificial Intelligence News reports that SoftBank's chief believes ASI will be here within 10 years and Malaysia launches its first AI-powered bank.

And 5 multiplied by 6 is 30.
